In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re


class ebay(object):
    def __init__(self, url, page):
        self.req = requests.session()
        self.get_page(url, page)

    def get_page(self, url, page):
        for i in range(1, page):
            response = self.req.get(url + "&_pgn=" + str(i))
            #time.sleep(2.4)
            print("Printed after 2.4 seconds.")
            if response.ok:
                soup = BeautifulSoup(response.content, 'lxml')
                self.get_product_detail(soup)
            else:
                print('Server responded:', response.status_code)

    def get_product_detail(self, source):
        if True:
            for i in source.find_all('a', class_='s-item__link', href=True):
                self.detail = self.req.get(i['href'])
                soup1 = BeautifulSoup(self.detail.content, 'lxml')
                title = str(
                    str(soup1.find('h1', attrs={
                        'id': 'itemTitle'
                    }).text).split('Details about')[-1]).strip()
                price = soup1.find("span", attrs={'itemprop': 'price'}).text
                location = soup1.find(
                    'span', attrs={
                        'itemprop': 'availableAtOrFrom'
                    }).text.strip()
                try:
                    shipping_cost = ''.join(
                        soup1.find('span', attrs={
                            'id': 'fshippingCost'
                        }).text.split())
                except:
                    shipping_cost = ''.join(
                        soup1.find('span', attrs={
                            'class': 'shp-sub-text'
                        }).text.split())

                sellername = soup1.find(
                    'span', attrs={
                        'class': 'mbg-nw'
                    }).text.strip()
                condition = soup1.find(
                    'div', attrs={
                        'itemprop': "itemCondition"
                    }).text.strip()
                """
                color = list()
                if soup1.find('select', attrs={'name': 'Color'}):
                    for i in soup1.find(
                            'select', attrs={
                                'name': 'Color'
                            }).find_all('option')[1:]:
                        color.append(i.text)
                """
                data = {
                    'Title': title,
                    'Price': price,
                    'Location': location,
                    'Shipping cost': shipping_cost,
                    'Sellername': sellername,
                    'Condition': condition
                }

                self.ebay_parser(data)

    def ebay_parser(self, data):
        print(data, "--------")
        try:
            p = re.compile(
                '"matchingVariationIds"\:\[(\w+)\],"displayName"\:"([^\\"]+)"}')
            symbol = str(
                str(
                    re.compile('currencySymbol":"[a-zA-Z]*[ ]*."').findall(
                        self.detail.text)[0]).split(':')[-1]).replace('"', '')
            product_data = list()
            new_data = {'Color': [], 'Price': []}
            for i in p.findall(self.detail.text):
                product_data.append(i)
            for i in range(len(product_data)):
                c = re.compile(str(product_data[i][0]) + '":\D*[0-9]*')
                for t in c.findall(self.detail.text):
                    new_data['Color'].append(product_data[i][1])
                    new_data['Price'].append(int(t.split(":")[-1]) / 100)
            if len(new_data['Color']) >= 1 and len(new_data['Price']) >= 1:
                for i in range(len(new_data['Color'])):
                    for k in range(len(new_data['Color'])):
                        data['Color'] = new_data['Color'][i]
                        data['Price'] = symbol + str(new_data['Price'][i])
                    self.csv_write(data)
            else:
                self.csv_write(data)
        except Exception as error:
            self.csv_write(data)
            

    def csv_write(self, data):
        local_data = list()
        csvfile = open('output12.csv', 'a', encoding='utf-8')
        writer = csv.writer(csvfile, delimiter=',')
        for i in data.values():
            local_data.append(i)
        writer.writerow(local_data)


ebay('https://www.ebay.com/sch/i.html?_nkw=robotic+vacuum+cleaner', 1)


Printed after 2.4 seconds.
{'Title': '3 IN 1 Smart Robot Vacuum Cleaner Auto Cleaning Microfiber Mop Floor Sweeper US', 'Price': 'US $12.99', 'Location': 'Shenzhen, China', 'Shipping cost': 'DoesnotshiptoTurkey', 'Sellername': 'dingfenglianhe01', 'Condition': 'New'} --------
{'Title': 'US Smart Robot Vacuum Cleaner Automatic Sweeper Floor Carpet Clean Rechargeable-', 'Price': 'US $10.79', 'Location': 'Hong Kong, China', 'Shipping cost': 'DoesnotshiptoTurkey', 'Sellername': 'fhew47', 'Condition': 'New'} --------
{'Title': 'bObsweep PetHair Robotic Vacuum Cleaner and Mop, Rouge', 'Price': 'C $379.83', 'Location': 'BC, Canada', 'Shipping cost': 'C$92.50(approx.US$63.88)', 'Sellername': 'medgames', 'Condition': 'New'} --------
{'Title': 'bObsweep bObi Pet Robotic Vacuum Cleaner, Silver - OP5727556SV', 'Price': 'C $531.18', 'Location': 'BC, Canada', 'Shipping cost': 'C$92.50(approx.US$63.88)', 'Sellername': 'medgames', 'Condition': 'New'} --------
{'Title': 'bObsweep WPP56001CH Pethair Plus